In [50]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import statistics
from sklearn.metrics.pairwise import cosine_similarity
import operator
from sklearn.metrics import r2_score
from tqdm import tqdm

In [2]:
text_db = pd.read_csv("text_db.csv", header = None)
text_db.head()

,0,1,2,3,4,5,6,7,8,9
0,0.6503,0.65,0.200000,0.200000,0.007800,0.007800,0.000000,0.093184,0.237405,0.50
1,0.5626,0.72,0.153846,0.307692,0.014890,0.011259,0.000000,0.169718,0.201980,0.40
2,0.4951,0.83,0.166667,0.000000,0.028309,0.012447,0.000000,0.170251,0.190485,0.50
3,0.6321,0.66,0.000000,0.250000,0.013009,0.018568,0.000000,0.223918,0.214968,0.25
4,0.6601,0.62,0.142857,0.214286,0.028585,0.004112,0.004621,0.106511,0.206026,0.50


In [6]:
text_db_arr = np.array([np.array(list(text_db.iloc[ind])) for ind in range(len(text_db))])
text_db_arr_X = text_db_arr[:,:-1]
text_db_arr_y = text_db_arr[:,-1]
X_train, X_test, y_train, y_test = train_test_split(text_db_arr_X, text_db_arr_y, test_size=0.15)

In [25]:
y_train[0]

1.0

In [39]:
word_db = pd.read_csv("word_db.csv", header = None)
word_db.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,0.025718,-0.004776,0.031789,0.018760,-0.015405,-0.078441,-0.002449,-0.008062,-0.001146,0.010007,...,0.017842,-0.041569,0.077341,0.012496,0.049400,-0.037332,0.001658,0.010653,-0.036526,-0.000000
1,0.018746,0.003426,0.020213,0.056091,-0.006876,-0.030144,0.021508,-0.038640,-0.008778,0.014135,...,0.000866,-0.069830,0.107995,0.041295,0.074002,-0.027547,-0.023377,-0.026141,-0.028869,-0.062423
2,0.030954,0.012660,-0.013010,0.058922,-0.015895,-0.004156,0.043479,-0.097470,0.013334,-0.000223,...,-0.012979,-0.054481,0.129445,0.047809,0.064443,-0.009311,-0.068905,-0.063734,-0.017192,-0.050362
3,0.029982,0.046834,-0.016939,0.034865,-0.004499,-0.025411,0.057243,-0.126531,0.015994,-0.012146,...,-0.063004,0.012827,0.101894,0.074595,0.044913,-0.022268,-0.048629,-0.074751,0.018179,-0.035168
4,0.023059,0.052791,-0.031716,0.051026,-0.040494,-0.007667,0.052596,-0.087467,-0.004326,0.018472,...,-0.052820,0.033877,0.062752,0.100702,0.030436,0.018350,-0.031551,-0.050780,0.061213,-0.059490


In [54]:
sentence_db = pd.read_csv("sentence_db.csv", header = None)
sentence_db.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.000000,0.0,0.213510,0.242857,0.0
1,0.000000,0.000000,0.024805,0.024805,0.0,0.000000,0.277500,0.0
2,0.000000,0.066667,0.000000,0.000000,0.0,0.000000,0.250000,1.0
3,0.076923,0.000000,0.000000,0.000000,0.0,0.138684,0.200000,1.0
4,0.000000,0.071429,0.000000,0.000000,0.0,0.000000,0.238462,1.0


In [74]:
def filter_similar(sorted_similarity):
    filtered_sorted_similarity_list = []
    first_similarity = sorted_similarity[0][1]
    if first_similarity < 0.7:
            return None
    for similar_el in sorted_similarity[:5]:
        similarity_value = similar_el[1]
        if similarity_value - first_similarity > 0.15 or similarity_value < 0.5:
            break
        filtered_sorted_similarity_list.append(similar_el)
    return filtered_sorted_similarity_list

In [77]:
%%time
prediction = []
def calculate_accuracy(database, debug = False):
    db_arr = np.array([np.array(list(database.iloc[ind])) for ind in range(len(database))])
    db_arr_X = db_arr[:,:-1]
    db_arr_y = db_arr[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(db_arr_X, db_arr_y, test_size=0.15)
    for test_vector_ind in tqdm(range(len(X_test))):
        test_vector_arr = np.array(X_test[test_vector_ind]).reshape(1,-1)
        test_vector_dict = {}
        for train_vector_ind in range(len(X_train)):
            train_vector = X_train[train_vector_ind]
            train_vector_arr = np.array(train_vector).reshape(1,-1)
            similarity = cosine_similarity(test_vector_arr, train_vector_arr)
            test_vector_dict[train_vector_ind] = similarity
        sorted_test_vector_dict = sorted(test_vector_dict.items(), key=operator.itemgetter(1), reverse = True)
        if debug: print(sorted_test_vector_dict[:5])
        sorted_test_vector_dict = filter_similar(sorted_test_vector_dict)
        if sorted_test_vector_dict:
            #normalization
            sum_similarity = 0
            for sort_element in sorted_test_vector_dict[:5]:
                sum_similarity += sort_element[1][0][0]

            normalized_similarity_list = []
            for sort_element in sorted_test_vector_dict[:5]:
                norm_sim = sort_element[1][0][0] / sum_similarity
                target_variable = y_train[sort_element[0]]
                normalized_similarity_list.append((norm_sim, target_variable))
            if debug: print(normalized_similarity_list)

            math_expectation = 0
            for norm_el in normalized_similarity_list:
                math_expectation += norm_el[0] * norm_el[1] 
            prediction.append(math_expectation)
            if debug:
                print(math_expectation)
                print()
        else:
            prediction.append(-0)
    if debug: print(prediction)
    print(r2_score(y_test,prediction))
    return y_test, prediction
true, prde = calculate_accuracy(word_db, debug = False)

100%|██████████| 161/161 [00:29<00:00,  5.60it/s]

0.3323447324414962
CPU times: user 29.1 s, sys: 1.64 s, total: 30.8 s
Wall time: 30.1 s


In [76]:
for ind in range(len(true)):
    print(true[ind], prde[ind])

-0.016937114279006273 0.037147815789919617
-0.06417838030013426 -0.053023487818136913
-0.0 0.0
-0.1821680964711896 -0.08260497626312667
0.04267569574674601 0.01851044493087302
-0.0498271657485375 -0.001730710804750536
0.0808426935619775 0.02953485034015923
0.0 0.040394379845914166
-0.022322688754111076 0.013428659073527255
-0.09703598921308991 0
0.12566841831706305 0.08512437076814984
0.06436482417090192 -0.0018977146092439626
-0.05036239072468087 0
-0.059700725838095714 -0.04193043052946486
-0.06417838030013426 -0.024094115572987032
-0.020173264535791668 -0.009184049644793961
-0.02347102930266891 0.00038567585967702253
-0.041565885079915786 0.03883499876865197
0.04267569574674601 0.03821420356232571
0.11218833335194173 -0.019769144338866167
-0.029057485077713455 -0.034943429881916024
0.04362399244811412 0.04609798803422002
-0.0498271657485375 -0.016740791191795088
-0.0 -0.013109895324978697
-0.052027412494165134 -0.05822656141287609
-0.027948878436470703 -0.03831275715476528
-0.0 0.01

In [41]:
y_test, prediction

(array([0.5, 0.6, 0.5]),
 [0.3913635811722645, 0.4890086799622072, 0.49048698519971473])

-2.6317163105056043